<a href="https://colab.research.google.com/github/delhiiitian/CrossSell/blob/main/nn_new2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
# Import packages
import numpy as np
import pandas as pd
# import matplotlib.pyplot as plt
# import seaborn as sns
from sklearn.compose import make_column_transformer
from sklearn.model_selection import train_test_split,cross_val_score,StratifiedKFold,KFold
from sklearn.preprocessing import  LabelEncoder,OrdinalEncoder,MinMaxScaler,OneHotEncoder
from keras.models import Sequential
import keras
from tensorflow.keras.utils import to_categorical 
from keras.layers import Dense, BatchNormalization, Dropout
from tensorflow.keras.optimizers import Adam, SGD, RMSprop, Adadelta, Adagrad, Adamax, Nadam, Ftrl
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.metrics import make_scorer, accuracy_score
# from bayes_opt import BayesianOptimization
from keras.layers import LeakyReLU
LeakyReLU = LeakyReLU(alpha=0.1)
import warnings
warnings.filterwarnings('ignore')
pd.set_option("display.max_columns", None)

In [21]:
traindf = pd.read_csv('train_full_new.csv',index_col=0)
testdf = pd.read_csv('test_full_new.csv',index_col=0)

testdf.drop(columns=['season'],inplace=True)

In [23]:
traindf = traindf[traindf['season'].isin(["a","u"]) ]

In [46]:
# one hot
# train = pd.get_dummies(traindf,columns=['edible-poisonous', 'cap-shape', 'cap-color',
#        'does-bruise-or-bleed', 'gill-attachment', 'gill-color',
#        'stem-color', 'has-ring', 'ring-type', 'habitat'],drop_first=True)
# test = pd.get_dummies(testdf,columns=['edible-poisonous', 'cap-shape', 'cap-color',
#        'does-bruise-or-bleed', 'gill-attachment', 'gill-color',
#        'stem-color', 'has-ring', 'ring-type', 'habitat',],drop_first=True)


train1, test1 = train_test_split(traindf, test_size=0.02,stratify=traindf['season'])

# train, val = train_test_split(train, test_size=0.2)

X_train,y_train = train1.loc[:, train1.columns != 'season'],train1['season']
X_test,y_test = test1.loc[:, test1.columns != 'season'],test1['season']

categorical_columns = ['edible-poisonous', 'cap-shape', 'cap-color',
       'does-bruise-or-bleed', 'gill-attachment', 'gill-color',
       'stem-color', 'has-ring', 'ring-type', 'habitat']

numerical_columns = ['cap-diameter','stem-height','stem-width']

column_trans = make_column_transformer((OneHotEncoder(),categorical_columns),(MinMaxScaler(),numerical_columns))


column_trans.fit(X_train)
X_train = column_trans.transform(X_train)
X_test = column_trans.transform(X_test)
testdata = column_trans.transform(testdf)
# # LE
# # prepare input data
# def prepare_inputs(X_train, X_test):
# 		oe = OrdinalEncoder()
# 		oe.fit(X_train)
# 		X_train_enc = oe.transform(X_train,)
# 		X_test_enc = oe.transform(X_test)
# 		return X_train_enc, X_test_enc

# prepare target
def prepare_targets(y_train, y_test):
		le = LabelEncoder()
		le.fit(y_train)
		y_train_enc = le.transform(y_train)
		y_test_enc = le.transform(y_test)
		return y_train_enc, y_test_enc

# # prepare input data
# # X_train_enc, X_test_enc = prepare_inputs(tr1, ts1)
# # prepare output data

mapping_dictionary = {"a":0,"u":1}
mappint_dictionary_reverse = {0:"a",1:"u"}
y_train = y_train.replace(mapping_dictionary,regex=True)
y_test = y_test.replace(mapping_dictionary,regex=True)
# y_train, y_test = prepare_targets(y_train, y_test)
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)


In [47]:
X_train.shape[1]

76

In [48]:
y_train

array([[1., 0.],
       [1., 0.],
       [1., 0.],
       ...,
       [1., 0.],
       [0., 1.],
       [1., 0.]], dtype=float32)

In [49]:
testdata.shape

(18321, 76)

In [59]:
model = Sequential()
model.add(Dense(128, activation='tanh', input_dim=X_train.shape[1]))
model.add(Dropout(0.1))
model.add(Dense(64, activation='tanh'))
model.add(Dropout(0.1))
# model.add(Dense(64, activation='relu'))
# model.add(Dropout(0.01))
model.add(Dense(2, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

In [60]:
model.fit(X_train, y_train, epochs=20)
model.evaluate(X_test, y_test, batch_size=16)

Epoch 1/20
1138/1138 [==============================] - 4s 3ms/step - loss: 0.6731 - accuracy: 0.5743
Epoch 2/20
1138/1138 [==============================] - 3s 3ms/step - loss: 0.6567 - accuracy: 0.5819
Epoch 3/20
1138/1138 [==============================] - 3s 3ms/step - loss: 0.6448 - accuracy: 0.5852
Epoch 4/20
1138/1138 [==============================] - 3s 3ms/step - loss: 0.6368 - accuracy: 0.5871
Epoch 5/20
1138/1138 [==============================] - 3s 3ms/step - loss: 0.6304 - accuracy: 0.5899
Epoch 6/20
1138/1138 [==============================] - 3s 3ms/step - loss: 0.6245 - accuracy: 0.5968
Epoch 7/20
1138/1138 [==============================] - 3s 3ms/step - loss: 0.6210 - accuracy: 0.5971
Epoch 8/20
1138/1138 [==============================] - 3s 3ms/step - loss: 0.6175 - accuracy: 0.5967
Epoch 9/20
1138/1138 [==============================] - 3s 3ms/step - loss: 0.6162 - accuracy: 0.5944
Epoch 10/20
1138/1138 [==============================] - 3s 3ms/step - loss: 0.613

[0.6036636233329773, 0.5860214829444885]

In [54]:
predictions = np.argmax(model.predict(testdata),axis=1)
pred_df = pd.DataFrame(predictions,columns=['season'])

In [56]:
pred_df = pred_df.replace({'season':mappint_dictionary_reverse})

In [58]:
pred_df.to_csv('final_v1.csv',index=None)